# We can convert the 6d to 7D to be more similar to Lockwood et al. 2022

In [1]:
# Convenient jupyter setup
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

In [1]:
import os
ex_path = "/Users/simon/new-orleans/data/kate_h08"
os.listdir(ex_path)

['minpr.63.nc',
 'fort.224.nc',
 'maxwvel.63.nc',
 'windDrag.173.nc',
 'fort.221.nc',
 'fort.73.nc',
 'fort.63.nc',
 'fort.22',
 'fort.14',
 'maxvel.63.nc',
 'fort.15',
 'fort.218',
 'fort.221',
 'fort.217',
 'fort.218.nc',
 'adcirc_log.txt',
 'maxele.63.nc',
 'fort.33',
 'fort.74.nc',
 'fort.64.nc',
 'fort.16',
 'fort.222.nc',
 'fort.217.nc',
 'fort.223.nc',
 'fort.224',
 'fort.223',
 'traj.nc',
 'fort.222']

In [18]:
file_names = ['fort.73.nc', 'fort.74.nc', 'fort.63.nc', 'fort.64.nc']
variables = [('pressure',), ('windx', 'windy'), ('zeta',), ('u-vel', 'v-vel')]

In [2]:
import netCDF4 as nc

In [14]:
for i in range(len(file_names)):
    print([x for x in nc.Dataset(os.path.join(ex_path, file_names[i])).variables])

['time', 'x', 'y', 'element', 'adcirc_mesh', 'neta', 'nvdll', 'max_nvdll', 'ibtypee', 'nbdv', 'nvel', 'nvell', 'max_nvell', 'ibtype', 'nbvv', 'depth', 'pressure']
['time', 'x', 'y', 'element', 'adcirc_mesh', 'neta', 'nvdll', 'max_nvdll', 'ibtypee', 'nbdv', 'nvel', 'nvell', 'max_nvell', 'ibtype', 'nbvv', 'depth', 'windx', 'windy']
['time', 'x', 'y', 'element', 'adcirc_mesh', 'neta', 'nvdll', 'max_nvdll', 'ibtypee', 'nbdv', 'nvel', 'nvell', 'max_nvell', 'ibtype', 'nbvv', 'depth', 'zeta']
['time', 'x', 'y', 'element', 'adcirc_mesh', 'neta', 'nvdll', 'max_nvdll', 'ibtypee', 'nbdv', 'nvel', 'nvell', 'max_nvell', 'ibtype', 'nbvv', 'depth', 'u-vel', 'v-vel']


In [25]:
for i in range(len(file_names)):
    for variable in variables[i]:
        print(variable)
        print(nc.Dataset(os.path.join(ex_path, file_names[i]))[variable].long_name)

pressure
air pressure at sea level
windx
e/w wind velocity
windy
n/s wind velocity
zeta
water surface elevation above geoid
u-vel
water column vertically averaged east/west velocity
v-vel
water column vertically averaged north/south velocity


In [21]:
ds = nc.Dataset(os.path.join(ex_path, file_names[0]))

In [24]:
ds["pressure"].long_name

'air pressure at sea level'

In [26]:
ds["time"].long_name

'model time'

In [27]:
ds["time"].units

'seconds since 2020-04-28 00:00:00'

In [31]:
ds["time"][:]

masked_array(data=[   4800.,    9600.,   14400.,   19200.,   24000.,
                     28800.,   33600.,   38400.,   43200.,   48000.,
                     52800.,   57600.,   62400.,   67200.,   72000.,
                     76800.,   81600.,   86400.,   91200.,   96000.,
                    100800.,  105600.,  110400.,  115200.,  120000.,
                    124800.,  129600.,  134400.,  139200.,  144000.,
                    148800.,  153600.,  158400.,  163200.,  168000.,
                    172800.,  177600.,  182400.,  187200.,  192000.,
                    196800.,  201600.,  206400.,  211200.,  216000.,
                    220800.,  225600.,  230400.,  235200.,  240000.,
                    244800.,  249600.,  254400.,  259200.,  264000.,
                    268800.,  273600.,  278400.,  283200.,  288000.,
                    292800.,  297600.,  302400.,  307200.,  312000.,
                    316800.,  321600.,  326400.,  331200.,  336000.,
                    340800.,  3456

## Add all this weird nc data into a single xarray dataset.

In [32]:
import xarray as xr

/Users/simon/new-orleans/env/lib/python3.8/site-packages/xarray/backends/cfgrib_.py:29: UserWarning: Failed to load cfgrib - most likely there is a problem accessing the ecCodes library. Try `import cfgrib` to get the full error message
  warnings.warn(


In [35]:
traj_ds = xr.open_dataset(os.path.join(ex_path, "traj.nc"))

In [36]:
traj_ds

<xarray.Dataset>
Dimensions:  (time: 56)
Coordinates:
  * time     (time) datetime64[ns] 2005-08-25 ... 2005-08-31T21:00:00
Data variables:
    lon      (time) float64 ...
    lat      (time) float64 ...

### Mess around with ADCIRC mesh

ds["adcirc_mesh"]

In [41]:
# triangular mesh elements stored in "element" variable
ds["element"][:]

masked_array(
  data=[[5189, 5434, 5190],
        [7845, 7777, 7844],
        [4551, 4806, 4807],
        ...,
        [ 997, 1079, 1078],
        [ 996, 1078, 1077],
        [1079, 1168, 1078]],
  mask=False,
  fill_value=999999,
  dtype=int32)

In [42]:
ds["element"][:].shape

(14761, 3)

Dimension: element, vertex, Units: mesh component [dimensionless]

In [48]:
xr.DataArray(ds["element"][:], dims=["element", "vertex"])

<xarray.DataArray (element: 14761, vertex: 3)>
array([[5189, 5434, 5190],
       [7845, 7777, 7844],
       [4551, 4806, 4807],
       ...,
       [ 997, 1079, 1078],
       [ 996, 1078, 1077],
       [1079, 1168, 1078]], dtype=int32)
Dimensions without coordinates: element, vertex

In [44]:
ds["depth"][:]  # depth at nodes

masked_array(data=[1., 1., 1., ..., 1., 1., 1.],
             mask=False,
       fill_value=1e+20)

In [46]:
ds["depth"][:].max(), ds["depth"][:].min() # can only get down to 1 meter of depth

(7987.0644531, 1.0)

In [47]:
ds["depth"][:].shape

(8303,)

In [49]:
xr.DataArray(ds["depth"][:], dims=["node"])

<xarray.DataArray (node: 8303)>
array([1., 1., 1., ..., 1., 1., 1.])
Dimensions without coordinates: node

### Let's look at the main variables

In [51]:
ds["pressure"][:].shape  # pressure at nodes

(252, 8303)

In [57]:
xr.Dataset(data_vars=dict(pressure=(["time", "node"], ds["pressure"][:])))

<xarray.Dataset>
Dimensions:   (time: 252, node: 8303)
Dimensions without coordinates: time, node
Data variables:
    pressure  (time, node) float64 10.33 10.33 10.33 10.33 ... 10.33 10.33 10.33

In [61]:
ds_list = []
for i in range(len(file_names)):
    for variable in variables[i]:
        print(variable)
        ds_list.append(xr.Dataset(data_vars={
        variable: (["time", "node"], nc.Dataset(os.path.join(ex_path, file_names[i]))[variable][:]),
        }))

pressure
windx
windy
zeta
u-vel
v-vel


In [62]:
ds_list[0]

<xarray.Dataset>
Dimensions:   (time: 252, node: 8303)
Dimensions without coordinates: time, node
Data variables:
    pressure  (time, node) float64 10.33 10.33 10.33 10.33 ... 10.33 10.33 10.33

In [65]:
xr.merge(ds_list)

<xarray.Dataset>
Dimensions:   (time: 252, node: 8303)
Dimensions without coordinates: time, node
Data variables:
    pressure  (time, node) float64 10.33 10.33 10.33 10.33 ... 10.33 10.33 10.33
    windx     (time, node) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    windy     (time, node) float64 0.0 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0 0.0 0.0
    zeta      (time, node) float64 3.793e-06 2.164e-05 ... -0.2057 -0.136
    u-vel     (time, node) float64 2.479e-05 2.382e-05 ... 0.06647 -0.001961
    v-vel     (time, node) float64 5.628e-06 1.081e-05 ... -0.01729 -0.00115

### How to convert from model time to datetime?